# inference on Docker Image


In [1]:
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

In [2]:
import os
%store -r training_job_name

no stored variable or alias training_job_name


In [3]:
training_job_name = 'bert2tweet-2020-07-08-07-58-27-895'
print(training_job_name)

bert2tweet-2020-07-08-07-58-27-895


In [4]:
container_image = '057716757052.dkr.ecr.us-west-2.amazonaws.com/sagemaker-tensorflow-serving:2.0.0-cpu'


In [5]:
from sagemaker.tensorflow.serving import Model

model = Model(model_data='s3://{}/{}/output/model.tar.gz'.format(bucket, training_job_name),
              role=role,
              framework_version='2.0.0',
              entry_point='inference.py',
              image=container_image
             ) # Elastic Inference does not yet support TF 2.1.0 as of sagemaker==1.56.1

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.


In [6]:
%%time

instance_type='ml.t2.medium'
deployed_model = model.deploy(initial_instance_count = 1,
                             instance_type = instance_type,
                             wait=True)


---------------!CPU times: user 1min 2s, sys: 9.76 s, total: 1min 12s
Wall time: 8min 46s


## Invoke the endpoint without custom Docker image

In [7]:
# tweet_bert_endpoint_name = 'sagemaker-tensorflow-serving-2020-07-09-00-37-50-247'
tweet_bert_endpoint_name = deployed_model.endpoint
print(tweet_bert_endpoint_name)

sagemaker-tensorflow-serving-2020-07-09-07-31-25-739


In [8]:
deployed_model.endpoint

'sagemaker-tensorflow-serving-2020-07-09-07-31-25-739'

In [9]:
reviews = [["This is not great!"]]
deployed_model.predict(reviews)

[[-1.25447786,
  -1.252895,
  -1.39534187,
  -1.05819726,
  7.12146568,
  -1.55694449,
  -2.16557837,
  -1.75193763,
  -2.2782464,
  -1.964239]]

In [11]:
import json
from sagemaker.tensorflow.serving import Predictor

predictor = Predictor(endpoint_name = tweet_bert_endpoint_name,
                      sagemaker_session = sess,
                      content_type = 'application/json',
                      model_name = 'saved_model',
                      model_version=0
                     )

In [36]:
# reviews = ["This is not great!", "You are awesome because you look great"]
# reviews = ["The weather is so nice causing me to be depressed!"]
reviews = ["hi"]
predicted_classes = predictor.predict(reviews)
predicted_classes[0]

[-1.30759752,
 -1.23688304,
 -1.4000138,
 -1.07528186,
 7.13767719,
 -1.54851651,
 -2.16409039,
 -1.7563585,
 -2.2845614,
 -1.96250415]

In [37]:
def show_top_N_label(score_list, topN):

    import numpy as np

    top_n_idx = np.argsort(score_list)[-topN:]
    top_n_values = [score_list[i] for i in top_n_idx]
    
    print(top_n_idx)


    top_n_idx_list = top_n_idx.tolist()
    top_n_idx_list.reverse()
    top_n_values = [score_list[i] for i in top_n_idx_list]    
    
    print(top_n_idx_list)
    print(top_n_values)

topN = 3    
show_top_N_label(predicted_classes[0], topN)


[1 3 4]
[4, 3, 1]
[7.13767719, -1.07528186, -1.23688304]


## Run on Boto3 Client

In [41]:
invoke_endpoint = 'sagemaker-tensorflow-serving-2020-07-09-07-31-25-739'

In [44]:
%%bash
echo {$invoke_endpoint}

{}


In [48]:
%%bash
aws sagemaker-runtime invoke-endpoint \
  --endpoint-name sagemaker-tensorflow-serving-2020-07-09-07-31-25-739 \
  --body '["This is great"]' \
  --content-type application/json \
  --accept application/json \
  results
echo $results

{
    "ContentType": "application/json",
    "InvokedProductionVariant": "AllTraffic"
}



In [51]:
payload = "This is great"
payload = '["' + payload + '"]'
print(payload)

["This is great"]


In [57]:
def try_test(num1):
    try:
        sum = num1 + 10
        return sum
    except:
        print("Error occured")
        return num1
    
result = try_test("2")
result

Error occured


'2'

In [ ]:
import json
import os
import io
import boto3
import csv

ENDPOINT_NAME = os.environ['ENDPOINT_NAME'] 
runtime = boto3.client('runtime.sagemaker')


def lambda_handler(event, context):
    # TODO implement
try:
print("Received event: " + json.dumps(event, indent=2))

data = json.loads(json.dumps(event))
payload = data['tweet']
payload = '["' + payload + '"]'
print("Final input: " ,payload)

print(payload)

response = runtime.invoke_endpoint(EndpointName = ENDPOINT_NAME,
                            ContentType = 'application/json',
                            Body = payload)

print(response)
result = json.loads(response['Body'].read().decode())
print(result)
return result
except:
print("An error occurred")
return data


# return {
#     'statusCode': 200,
#     'body': json.dumps('Hello from Lambda!')
# }
